<H2>Detecting nodes within a distance that may occur due to unjoined mesh</H2> 

In [1]:
import pandas as pd
import numpy as np
import itertools   
import time
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.19.0")
if not lusas.existsDatabase():
    Exception("A model must be open before running this code")
db = lusas.database()

Define the distance below which we want to detect adjacent nodes

In [2]:
TOLERANCE = 0.1

## Option 1, using itertools to generate the necessary comparison combinations

Note, the limit of 100 nodes because this code is very slow for large number of nodes

In [3]:
# Get all the nodes in the model
nodes = lusas.database().getObjects("Nodes")

if len(nodes) > 100:
    assert False, "This will take a very long time"

start = time.time()
no_comparisons = 0

# Use itertools to generate the comparison of each node with each other node
# No of comparisons will be n? where the ? indicates sumation of all integers fron n to 0
for a, b in itertools.combinations(nodes, 2):
    no_comparisons+=1
    delta = np.sqrt( (a.getX() - b.getX())**2 + (a.getY() - b.getY())**2 + (a.getZ() - b.getZ())**2 )
    if delta < TOLERANCE:
        print(f"Nodes {a.getID()} and {b.getID()} are within {delta:.3f}")

print(f"Comparison time = {time.time() - start:.2f} seconds for {no_comparisons} comparisons")


Nodes 6 and 26 are within 0.071
Nodes 7 and 27 are within 0.071
Nodes 8 and 28 are within 0.071
Nodes 9 and 29 are within 0.071
Nodes 10 and 30 are within 0.071
Comparison time = 6.72 seconds for 435 comparisons


## Option 1a - Reduce the calls to the LPI

Same approach as above here but first we compile a list of node coordinates. 
This avoids many repeated calls to get node coordinates

In [4]:

start = time.time()
no_comparisons = 0

positions  = np.array([[n.getX(), n.getY(), n.getZ(), n.getID()] for n in nodes])
for a, b in itertools.combinations(positions, 2):
    no_comparisons+=1
    delta = np.sqrt( (a[0] - b[0])**2 + (a[1] - b[1])**2 + (a[2] - b[2])**2 )
    if delta < TOLERANCE:
        print(f"Nodes {a[3]} and {b[3]} are within {delta:.3f}")

print(f"Comparison time = {time.time() - start:.2f} seconds for {no_comparisons} comparisons")

Nodes 6.0 and 26.0 are within 0.071
Nodes 7.0 and 27.0 are within 0.071
Nodes 8.0 and 28.0 are within 0.071
Nodes 9.0 and 29.0 are within 0.071
Nodes 10.0 and 30.0 are within 0.071
Comparison time = 0.29 seconds for 435 comparisons


As you can see repeated calls to the LPI functions can have a dramatic affect on performance and should be avoided where posible 

## Option 2 using numpy arrays

In [5]:
start = time.time()
ids = np.array([n.getID() for n in nodes])
positions  = np.array([[n.getX(), n.getY(), n.getZ()] for n in nodes])

# Relative positions of each node 
rel = positions[None, :, :] - positions[:, None, :]

# distances of each node from the x,y,z relative offsets
distances = np.sqrt(np.square(rel).sum(axis=2))


for i in range(0, len(distances)):
    a = np.where(distances[i] < TOLERANCE)
    if len(a[0]) == 2 and a[0][0] != i:
        print(f"Nodes {ids[a[0][0]]} and {ids[a[0][1]]} are within {distances[i][a[0][0]]:.3f}")


print(f"Comparison time = {time.time() - start:.2f} seconds using numpy arrays")

Nodes 6 and 26 are within 0.071
Nodes 7 and 27 are within 0.071
Nodes 8 and 28 are within 0.071
Nodes 9 and 29 are within 0.071
Nodes 10 and 30 are within 0.071
Comparison time = 0.30 seconds using numpy arrays
